In [2]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from datasets import load_dataset, concatenate_datasets
from sklearn.metrics import classification_report

In [3]:
hiner = load_dataset('cfilt/HiNER-original')
train_data = hiner['train']
val_data = hiner['validation']
test_data = hiner['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset hi_ner_config downloaded and prepared to /root/.cache/huggingface/datasets/cfilt___hi_ner_config/HiNER/0.0.2/c2bf095b51bde10ac392c9203c0fcdd1d7c47d2b03b6b455bf277f1afd7feed0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
combined_data = concatenate_datasets([train_data, val_data])

In [5]:
tokens = combined_data['tokens']
ner_tags = combined_data['ner_tags']

In [7]:
all_tokens = [token for sublist in tokens for token in sublist]
all_tags = [tag for sublist in ner_tags for tag in sublist]
token_vocab = sorted(set(all_tokens))
tag_vocab = sorted(set(all_tags))
# tag_vocab.append(int(23))

In [8]:
token_to_id = {token: i for i, token in enumerate(token_vocab)}
tag_to_id = {tag: i for i, tag in enumerate(tag_vocab)}

In [10]:
# Preprocess the training data
train_tokens = train_data['tokens']
train_tags = train_data['ner_tags']
train_tokens_ids = [[token_to_id[token] for token in sublist] for sublist in train_tokens]
train_tags_ids = [[tag_to_id[tag] for tag in sublist] for sublist in train_tags]
max_length = max(len(seq) for seq in train_tokens_ids)  # Calculate the maximum length
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=max_length, padding='post')
train_tags_ids = pad_sequences(train_tags_ids, maxlen=max_length, padding='post')
train_tags_ids = to_categorical(train_tags_ids, num_classes=len(tag_vocab))

In [11]:
# Preprocess the validation data
val_tokens = val_data['tokens']
val_tags = val_data['ner_tags']
val_tokens_ids = [[token_to_id.get(token, token_to_id.get('<unk>', 22)) for token in sublist] for sublist in val_tokens]
val_tags_ids = [[tag_to_id.get(tag, tag_to_id.get('<unk>', 22)) for tag in sublist] for sublist in val_tags]
val_tokens_ids = pad_sequences(val_tokens_ids, maxlen=max_length, padding='post')
val_tags_ids = pad_sequences(val_tags_ids, maxlen=max_length, padding='post')
val_tags_ids = to_categorical(val_tags_ids, num_classes=len(tag_vocab))

In [12]:
# Preprocess the test data
test_tokens = test_data['tokens']
test_tags = test_data['ner_tags']
test_tokens_ids = [[token_to_id.get(token, token_to_id.get('<unk>', 22)) for token in sublist] for sublist in test_tokens]
test_tags_ids = [[tag_to_id.get(tag, token_to_id.get('<unk>', 22)) for tag in sublist] for sublist in test_tags]
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=max_length, padding='post')
test_tags_ids = pad_sequences(test_tags_ids, maxlen=max_length, padding='post')
test_tags_ids = to_categorical(test_tags_ids, num_classes=len(tag_vocab))

In [13]:
model = tf.keras.Sequential([
    layers.Embedding(input_dim=len(token_vocab), output_dim=64),
    layers.Bidirectional(layers.GRU(128, return_sequences=True)),
    layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(tag_vocab), activation='softmax')
])

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [15]:
model.fit(train_tokens_ids, train_tags_ids, validation_data=(val_tokens_ids, val_tags_ids), epochs=5, batch_size=128)

Epoch 1/5
593/593 [==============================] - 617s 1s/step - loss: 0.1151 - accuracy: 0.9794 - precision: 0.9904 - recall: 0.9618 - val_loss: 0.0280 - val_accuracy: 0.9923 - val_precision: 0.9966 - val_recall: 0.9891
Epoch 2/5
593/593 [==============================] - 616s 1s/step - loss: 0.0187 - accuracy: 0.9948 - precision: 0.9969 - recall: 0.9932 - val_loss: 0.0173 - val_accuracy: 0.9951 - val_precision: 0.9965 - val_recall: 0.9942
Epoch 3/5
593/593 [==============================] - 581s 980ms/step - loss: 0.0117 - accuracy: 0.9965 - precision: 0.9973 - recall: 0.9960 - val_loss: 0.0159 - val_accuracy: 0.9955 - val_precision: 0.9963 - val_recall: 0.9949
Epoch 4/5
593/593 [==============================] - 597s 1s/step - loss: 0.0091 - accuracy: 0.9972 - precision: 0.9976 - recall: 0.9969 - val_loss: 0.0154 - val_accuracy: 0.9956 - val_precision: 0.9963 - val_recall: 0.9952
Epoch 5/5
593/593 [==============================] - 584s 985ms/step - loss: 0.0075 - accuracy: 0.997

In [15]:
test_loss, test_accuracy = model.evaluate(test_tokens_ids, test_tags_ids)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

677/677 [==============================] - 50s 74ms/step - loss: 0.0160 - accuracy: 0.9957
Test Loss: 0.01596505008637905
Test Accuracy: 0.9957199096679688


In [16]:
model.save('NLP_NER_BiGRU.h5')

In [17]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions on test data
predictions = model.predict(test_tokens_ids)

# Convert predictions from categorical to label encoded
pred_tags_ids = np.argmax(predictions, axis=-1)

# Convert test tags from one-hot encoded to label encoded
true_tags_ids = np.argmax(test_tags_ids, axis=-1)

# Flatten both arrays
pred_tags_ids_flat = pred_tags_ids.flatten()
true_tags_ids_flat = true_tags_ids.flatten()

# Use inverse_transform to map predicted and true labels back to their original form.
# You can skip this step if you want the classification report for the encoded labels.
pred_tags = [tag_vocab[i] for i in pred_tags_ids_flat]
true_tags = [tag_vocab[i] for i in true_tags_ids_flat]

# Generate classification report
report = classification_report(true_tags, pred_tags, output_dict=True)

report = classification_report(true_tags, pred_tags)
print(report)

677/677 [==============================] - 64s 93ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   3455846
           1       0.65      0.72      0.68       369
           2       0.96      0.91      0.93      1190
           3       0.82      0.46      0.59       179
           4       0.97      0.92      0.94     40072
           5       0.64      0.58      0.61      1065
           6       0.79      0.71      0.75      4638
           7       0.83      0.78      0.80      5351
           8       0.88      0.79      0.83      7495
           9       0.75      0.81      0.78       230
          10       0.84      0.80      0.82      3645
          11       0.00      0.00      0.00        26
          12       0.56      0.74      0.64       231
          13       0.93      0.98      0.96       603
          14       0.76      0.50      0.60       171
          15       0.87      0.73      0.80      4731
          16       0.42 